## Telco Customer Churn: Exploratory Data Analysis

### 1. Understand the data 

**Load Libraries**

In [17]:
import pandas as pd
import openpyxl

**Load Dataset**

In [28]:
df = pd.read_csv(r"C:\Users\linto\Code\churn-x\telcoData\Telco_customer_churn.csv")

### 2. Data Cleaning

### 3. Univariate Analysis

### 4. Bivariate/Multivariate Analysis

### 5. Feature Engineering Insights

### 6. Data Distribution & Assumptions

### 7. Key Insights & Hypothesis